In [21]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#Check Ethan Rosenthal github

In [143]:
class tensor_fact(nn.Module):
    def __init__(self,n_pat=10,n_meas=5,n_t=25,l_dim=2,n_u=2,n_w=3):
        super(tensor_fact,self).__init__()
        self.n_pat=n_pat
        self.n_meas=n_meas
        self.n_t=n_t
        self.l_dim=l_dim
        self.n_u=n_u
        self.n_w=n_w
        self.pat_lat=nn.Embedding(n_pat,l_dim) #sparse gradients ?
        self.pat_lat.weight=nn.Parameter(torch.randn([n_pat,l_dim]))
        self.meas_lat=nn.Embedding(n_meas,l_dim)
        self.meas_lat.weight=nn.Parameter(torch.randn([n_meas,l_dim]))
        self.time_lat=nn.Embedding(n_t,l_dim).double()
        self.beta_u=torch.nn.Parameter(torch.randn([n_u,l_dim],requires_grad=True).double())
        self.beta_w=torch.nn.Parameter(torch.randn([n_w,l_dim],requires_grad=True).double())
    def forward(self,idx_pat,idx_meas,idx_t,cov_u,cov_w):
        pred=((self.pat_lat(idx_pat)+torch.mm(cov_u,self.beta_u))*(self.meas_lat(idx_meas))*(self.time_lat(idx_t)+torch.mm(cov_w,self.beta_w))).sum(1) 
        return(pred)
        
        

In [127]:
class TensorFactDataset(Dataset):
    def __init__(self,csv_file_serie="lab_short_tensor_train.csv",file_path="~/Documents/Data/Full_MIMIC/",transform=None):
        self.lab_short=pd.read_csv(file_path+csv_file_serie)
        self.length=len(self.lab_short.index)
        self.pat_num=self.lab_short["UNIQUE_ID"].nunique()
        self.cov_values=[chr(i) for i in range(ord('A'),ord('A')+18)]
        self.time_values=["TIME_STAMP","TIME_SQ"]
        #print(self.lab_short.dtypes)
        self.tensor_mat=self.lab_short.as_matrix()
    def __len__(self):
        return self.length
    def __getitem__(self,idx):
        #ind_sparse=self.sparse_tens.getrow(idx)
        
        #print(self.lab_short["VALUENUM"].iloc[idx].values)
        #return(self.lab_short.iloc[idx].as_matrix())
        return(self.tensor_mat[idx,:])
        #return([torch.from_numpy(self.lab_short.iloc[idx][["UNIQUE_ID","LABEL_CODE","TIME_STAMP"]].astype('int64').as_matrix()),torch.from_numpy(self.lab_short.iloc[idx][self.cov_values].as_matrix()),torch.from_numpy(self.lab_short.iloc[idx][self.time_values].astype('float64').as_matrix()),torch.tensor(self.lab_short["VALUENUM"].iloc[idx],dtype=torch.double)])
    

In [144]:
#With Adam optimizer
import time 

tens_dataset=TensorFactDataset()
dataloader = DataLoader(tens_dataset, batch_size=100000,shuffle=True)

mod=tensor_fact(n_pat=tens_dataset.pat_num,n_meas=30,n_t=101,n_u=18,n_w=1)
mod.double()

optimizer=torch.optim.Adam(mod.parameters(), lr=0.001)
criterion = nn.MSELoss(size_average=False)#
epochs_num=1

for epoch in range(epochs_num):
    print("EPOCH : "+str(epoch))
    for i_batch,sampled_batch in enumerate(dataloader):
        
        print(i_batch)
        starttime=time.time()
        indexes=sampled_batch[:,1:4].to(torch.long)
        cov_u=sampled_batch[:,4:22]
        cov_w=sampled_batch[:,3].unsqueeze(1)
        target=sampled_batch[:,-1]
        
        
        optimizer.zero_grad()
        #preds=mod.forward(sampled_batch[0][:,0],sampled_batch[0][:,1],sampled_batch[0][:,2],sampled_batch[1],sampled_batch[2])
       
        preds=mod.forward(indexes[:,0],indexes[:,1],indexes[:,2],cov_u,cov_w)
        loss=criterion(preds,target)
        loss.backward()
        optimizer.step()
        print(time.time()-starttime)
        
        

EPOCH : 0
0
0.5869932174682617
1
0.5800011157989502
2


KeyboardInterrupt: 

In [42]:
a=TensorFactDataset()
b=a.__getitem__(3)
b

array([ 3.,  0.,  0., 75.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  9.])

In [46]:
indexes=torch.Tensor(b[:3]).int()
indexes

tensor([ 3,  0,  0], dtype=torch.int32)

In [35]:
csv_file_serie="lab_short_tensor.csv"
file_path="~/Documents/Data/Full_MIMIC/"
lab_short=pd.read_csv(file_path+csv_file_serie)

In [2]:
idx=[174729,3]
a=lab_short.iloc[idx]["VALUENUM"]

NameError: name 'lab_short' is not defined

In [282]:
lab_short.iloc[3]["VALUENUM"]

9.0

In [30]:
a=np.array([1,2,3])

In [31]:
torch.Tensor(a)

tensor([ 1.,  2.,  3.])

In [79]:
torch.randn(3).double().dtype

torch.float64

64515

In [84]:
lab_dim=3
lab_dim*(lab_short["UNIQUE_ID"].nunique()+30+101)/len(lab_short.index)

0.022705276605628894

In [85]:
len(lab_short.index)/(lab_short["UNIQUE_ID"].nunique()*30*101)

0.043873822209581304

In [90]:
np.sqrt(0.5)

0.7071067811865476

In [125]:
mod=tensor_fact(n_pat=tens_dataset.pat_num,n_meas=30,n_t=101,n_u=18,n_w=1)
for i in enumerate(mod.parameters()):
    print(i)

(0, Parameter containing:
tensor([[ 0.3364,  2.3571],
        [-1.5298,  2.7330],
        [ 1.1631,  0.3089],
        [-2.1276, -0.3360],
        [-0.6021, -0.0876],
        [ 0.4438, -0.0219],
        [-1.8984,  1.8621],
        [ 0.1408,  0.7330],
        [ 0.3796, -1.3308],
        [-0.5094, -1.4694],
        [ 0.5451,  0.5626],
        [ 1.6695,  1.3612],
        [ 1.1912, -0.8589],
        [ 0.7064, -1.7086],
        [ 0.3451, -0.3606],
        [-0.3710, -1.2147],
        [ 1.3785, -0.9152],
        [ 0.2956, -2.4411]], dtype=torch.float64))
(1, Parameter containing:
tensor([[ 0.9371,  0.1345]], dtype=torch.float64))
(2, Parameter containing:
tensor([[ 3.0396e-01,  1.7614e+00],
        [ 2.2776e-01,  5.4416e-01],
        [-1.0709e+00,  1.0852e+00],
        ...,
        [-1.2617e-01,  6.6292e-01],
        [-6.0991e-01,  2.3274e-01],
        [-5.7461e-01, -2.3834e-01]]))
(3, Parameter containing:
tensor([[-0.8869,  1.0772],
        [-0.4788,  1.6747],
        [-2.0322, -0.5766],
   

In [148]:
a=nn.Embedding(n_meas,l_dim).double()
a.weight

Parameter containing:
tensor([[ 0.5258, -0.0692, -0.4372, -0.6689,  1.9913],
        [-1.4903, -2.2111,  0.9129, -0.2722,  1.8234],
        [-0.1660, -0.2575,  0.6380, -0.7633, -0.9242],
        [ 2.7356,  0.2454, -1.1247,  0.4840,  1.6507],
        [ 1.3054,  0.3608, -0.3589,  1.2875,  0.3299]], dtype=torch.float64)

In [149]:
a.weight=nn.Parameter(0.5*torch.randn([5,5]))

In [150]:
a.weight

Parameter containing:
tensor([[ 0.6721, -0.3864,  0.6945, -0.6208, -0.3603],
        [-0.7730,  0.2382, -0.3704, -0.0439, -0.5443],
        [ 0.2399,  0.3886, -0.3603,  0.0543,  1.0006],
        [ 0.9537,  0.1054, -0.3608,  0.0165,  0.5242],
        [-0.1822, -0.3476,  1.1672, -0.0865, -0.5477]])

In [140]:
a.double()

Embedding(5, 5)